In [1]:
# !pip3 install --upgrade --quiet  langchain langchain-community langchainhub gpt4all chromadb 
# !pip3 install unstructured
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import GPT4AllEmbeddings
import os

/Users/rohitmalhotra/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
directories = ["About Scottie", "Buggy News", "academic_calendars", "history_of_cmu", "history_of_scs", "Kiltie Band", "lti_faculty", "lti_programs", "program_handbooks", "Tartan Facts"]

In [3]:
docs = []
for i in directories:
    loader = DirectoryLoader(i, glob="*.txt", exclude="annotation.txt")
    docs.append(loader.load())

In [4]:
all_splits = []
for i in docs:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    all_splits.append(text_splitter.split_documents(i))

In [5]:
for i in all_splits:
    vectorstore = Chroma.from_documents(documents=i, embedding=GPT4AllEmbeddings(), persist_directory="db/")

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522
bert_load_from_file: g